# 농산물 가격 예측을 위한 AI 모델 개발 
- '2024 농산물 가격 예측 AI 경진대회'는 데이터와 AI 기술을 활용하여 농산물 가격 예측 능력을 향상시키는 것을 목표로 합니다.<br>  이 대회는 농업 분야의 복잡한 시계열 데이터를 효율적으로 분석하고 예측할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br> <br>
- 이 대회의 궁극적 목적은 참가자들의 시계열 데이터 분석 및 예측 역량을 강화하고, <br> AI 기술이 실제 농산물 가격 예측과 관련 정책 결정에 어떻게 기여할 수 있는지 탐구하는 것입니다. 

# Import Library

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from types import SimpleNamespace
from sklearn.preprocessing import MinMaxScaler
import os

# Hyperparameter Setting

In [2]:
config = {
    "learning_rate": 2e-4,
    "epoch": 100,
    "batch_size": 64,
    "hidden_size": 64,
    "num_layers": 2,
    "output_size": 3
}

CFG = SimpleNamespace(**config)

품목_리스트 = ['건고추', '사과', '감자', '배', '깐마늘(국산)', '무', '상추', '배추', '양파', '대파']

# Define Function for Feature Engineering
- 타겟의 필터 조건을 제외한 메타데이터의 필터 조건은 참가자들 각자의 기준에 맞춰 자유롭게 사용가능 
- 밑의 필터 조건은 임의로 제공하는 예시

In [10]:

def process_data(raw_file, 품목명, scaler=None):
    raw_data = pd.read_csv(raw_file)

    # 타겟 및 메타데이터 필터 조건 정의
    conditions = {
    '감자': {
        'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상')
    },
    '건고추': {
        'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품')
    },
    '깐마늘(국산)': {
        'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품')
    },
    '대파': {
        'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상')
    },
    '무': {
        'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상')
    },
    '배추': {
        'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상')
    },
    '사과': {
        'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품')
    },
    '상추': {
        'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품')
    },
    '양파': {
        'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상')
    },
    '배': {
        'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품')
    }
    }

    # 타겟 데이터 필터링
    raw_품목 = raw_data[raw_data['품목명'] == 품목명]
    target_mask = conditions[품목명]['target'](raw_품목)
    filtered_data = raw_품목[target_mask]

    # 다른 품종에 대한 파생변수 생성
    other_data = raw_품목[~target_mask]
    unique_combinations = other_data[['품종명', '거래단위', '등급']].drop_duplicates()
    for _, row in unique_combinations.iterrows():
        품종명, 거래단위, 등급 = row['품종명'], row['거래단위'], row['등급']
        mask = (other_data['품종명'] == 품종명) & (other_data['거래단위'] == 거래단위) & (other_data['등급'] == 등급)
        temp_df = other_data[mask]
        for col in ['평년 평균가격(원)', '평균가격(원)']:
            new_col_name = f'{품종명}_{거래단위}_{등급}_{col}'
            filtered_data = filtered_data.merge(temp_df[['시점', col]], on='시점', how='left', suffixes=('', f'_{new_col_name}'))
            filtered_data.rename(columns={f'{col}_{new_col_name}': new_col_name}, inplace=True)


    # 공판장 데이터 처리
 

    # 도매 데이터 처리


    # 수치형 컬럼 처리
    numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
    filtered_data = filtered_data[['시점'] + list(numeric_columns)]
    filtered_data[numeric_columns] = filtered_data[numeric_columns].fillna(0)

    # 정규화 적용
    if scaler is None:
        scaler = MinMaxScaler()
        filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
    else:
        filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])

    return filtered_data, scaler


# Define Custom Dataset Class

In [4]:
class AgriculturePriceDataset(Dataset):
    def __init__(self, dataframe, window_size=9, prediction_length=3, is_test=False):
        self.data = dataframe
        self.window_size = window_size
        self.prediction_length = prediction_length
        self.is_test = is_test

        self.price_column = [col for col in self.data.columns if '평균가격(원)' in col and len(col.split('_')) == 1][0]
        self.numeric_columns = self.data.select_dtypes(include=[np.number]).columns.tolist()

        self.sequences = []
        if not self.is_test:
            for i in range(len(self.data) - self.window_size - self.prediction_length + 1):
                x = self.data[self.numeric_columns].iloc[i:i+self.window_size].values
                y = self.data[self.price_column].iloc[i+self.window_size:i+self.window_size+self.prediction_length].values
                self.sequences.append((x, y))
        else:
            self.sequences = [self.data[self.numeric_columns].values]

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if not self.is_test:
            x, y = self.sequences[idx]
            return torch.FloatTensor(x), torch.FloatTensor(y)
        else:
            return torch.FloatTensor(self.sequences[idx])

# Define Model Architecture and Training Functions

In [5]:

class PricePredictionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PricePredictionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
    return total_loss / len(test_loader)

# Train Models and Generate Predictions

In [11]:
품목별_predictions = {}
품목별_scalers = {}

pbar_outer = tqdm(품목_리스트, desc="품목 처리 중", position=0)
for 품목명 in pbar_outer:
    pbar_outer.set_description(f"품목별 전처리 및 모델 학습 -> {품목명}")
    train_data, scaler = process_data("./train/train.csv", 
                              품목명)
    품목별_scalers[품목명] = scaler
    dataset = AgriculturePriceDataset(train_data)

    # 데이터를 train과 validation으로 분할
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
    
    train_loader = DataLoader(train_data, CFG.batch_size, shuffle=True)
    val_loader = DataLoader(val_data, CFG.batch_size, shuffle=False)

    input_size = len(dataset.numeric_columns)
    
    model = PricePredictionLSTM(input_size, CFG.hidden_size, CFG.num_layers, CFG.output_size)
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), CFG.learning_rate)
    
    best_val_loss = float('inf')
    os.makedirs('models', exist_ok=True)

    for epoch in range(CFG.epoch):
        train_loss = train_model(model, train_loader, criterion, optimizer, CFG.epoch)
        val_loss = evaluate_model(model, val_loader, criterion)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'models/best_model_{품목명}.pth')
        
        print(f'Epoch {epoch+1}/{CFG.epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    print(f'Best Validation Loss for {품목명}: {best_val_loss:.4f}')
    
    품목_predictions = []

    ### 추론 
    pbar_inner = tqdm(range(25), desc="테스트 파일 추론 중", position=1, leave=False)
    for i in pbar_inner:
        test_file = f"./test/TEST_{i:02d}.csv"
        
        test_data, _ = process_data(test_file, 품목명, scaler=품목별_scalers[품목명])
        test_dataset = AgriculturePriceDataset(test_data, is_test=True)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        model.eval()
        predictions = []
        with torch.no_grad():
            for batch in test_loader:
                output = model(batch)
                predictions.append(output.numpy())
        
        predictions_array = np.concatenate(predictions)

        # 예측값을 원래 스케일로 복원
        price_column_index = test_data.columns.get_loc(test_dataset.price_column)
        predictions_reshaped = predictions_array.reshape(-1, 1)
        
        # 가격 열에 대해서만 inverse_transform 적용
        price_scaler = MinMaxScaler()
        price_scaler.min_ = 품목별_scalers[품목명].min_[price_column_index]
        price_scaler.scale_ = 품목별_scalers[품목명].scale_[price_column_index]
        predictions_original_scale = price_scaler.inverse_transform(predictions_reshaped)
        #print(predictions_original_scale)
        
        if np.isnan(predictions_original_scale).any():
            pbar_inner.set_postfix({"상태": "NaN"})
        else:
            pbar_inner.set_postfix({"상태": "정상"})
            품목_predictions.extend(predictions_original_scale.flatten())

            
    품목별_predictions[품목명] = 품목_predictions
    pbar_outer.update(1)


품목 처리 중:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/100, Train Loss: 0.7612, Val Loss: 0.7344
Epoch 2/100, Train Loss: 0.7529, Val Loss: 0.7271
Epoch 3/100, Train Loss: 0.7485, Val Loss: 0.7198
Epoch 4/100, Train Loss: 0.7410, Val Loss: 0.7125
Epoch 5/100, Train Loss: 0.7288, Val Loss: 0.7052
Epoch 6/100, Train Loss: 0.7222, Val Loss: 0.6978
Epoch 7/100, Train Loss: 0.7133, Val Loss: 0.6902
Epoch 8/100, Train Loss: 0.7056, Val Loss: 0.6825
Epoch 9/100, Train Loss: 0.7032, Val Loss: 0.6745
Epoch 10/100, Train Loss: 0.6918, Val Loss: 0.6661
Epoch 11/100, Train Loss: 0.6827, Val Loss: 0.6574
Epoch 12/100, Train Loss: 0.6757, Val Loss: 0.6482
Epoch 13/100, Train Loss: 0.6681, Val Loss: 0.6384
Epoch 14/100, Train Loss: 0.6535, Val Loss: 0.6278
Epoch 15/100, Train Loss: 0.6410, Val Loss: 0.6164
Epoch 16/100, Train Loss: 0.6327, Val Loss: 0.6040
Epoch 17/100, Train Loss: 0.6176, Val Loss: 0.5904
Epoch 18/100, Train Loss: 0.6078, Val Loss: 0.5752
Epoch 19/100, Train Loss: 0.5932, Val Loss: 0.5583
Epoch 20/100, Train Loss: 0.5722, Val Lo

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/100, Train Loss: 0.7907, Val Loss: 0.7963
Epoch 2/100, Train Loss: 0.7843, Val Loss: 0.7893
Epoch 3/100, Train Loss: 0.7740, Val Loss: 0.7823
Epoch 4/100, Train Loss: 0.7681, Val Loss: 0.7751
Epoch 5/100, Train Loss: 0.7591, Val Loss: 0.7678
Epoch 6/100, Train Loss: 0.7529, Val Loss: 0.7604
Epoch 7/100, Train Loss: 0.7485, Val Loss: 0.7527
Epoch 8/100, Train Loss: 0.7357, Val Loss: 0.7447
Epoch 9/100, Train Loss: 0.7260, Val Loss: 0.7364
Epoch 10/100, Train Loss: 0.7187, Val Loss: 0.7279
Epoch 11/100, Train Loss: 0.7123, Val Loss: 0.7189
Epoch 12/100, Train Loss: 0.7035, Val Loss: 0.7094
Epoch 13/100, Train Loss: 0.6941, Val Loss: 0.6992
Epoch 14/100, Train Loss: 0.6799, Val Loss: 0.6882
Epoch 15/100, Train Loss: 0.6702, Val Loss: 0.6762
Epoch 16/100, Train Loss: 0.6578, Val Loss: 0.6630
Epoch 17/100, Train Loss: 0.6450, Val Loss: 0.6485
Epoch 18/100, Train Loss: 0.6260, Val Loss: 0.6323
Epoch 19/100, Train Loss: 0.6126, Val Loss: 0.6142
Epoch 20/100, Train Loss: 0.5895, Val Lo

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

...

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/100, Train Loss: 0.4979, Val Loss: 0.4905
Epoch 2/100, Train Loss: 0.4855, Val Loss: 0.4816
Epoch 3/100, Train Loss: 0.4735, Val Loss: 0.4726
Epoch 4/100, Train Loss: 0.4659, Val Loss: 0.4632
Epoch 5/100, Train Loss: 0.4471, Val Loss: 0.4537
Epoch 6/100, Train Loss: 0.4458, Val Loss: 0.4438
Epoch 7/100, Train Loss: 0.4334, Val Loss: 0.4333
Epoch 8/100, Train Loss: 0.4173, Val Loss: 0.4222
Epoch 9/100, Train Loss: 0.4091, Val Loss: 0.4102
Epoch 10/100, Train Loss: 0.3939, Val Loss: 0.3975
Epoch 11/100, Train Loss: 0.3852, Val Loss: 0.3843
Epoch 12/100, Train Loss: 0.3640, Val Loss: 0.3699
Epoch 13/100, Train Loss: 0.3502, Val Loss: 0.3544
Epoch 14/100, Train Loss: 0.3340, Val Loss: 0.3372
Epoch 15/100, Train Loss: 0.3123, Val Loss: 0.3186
Epoch 16/100, Train Loss: 0.2971, Val Loss: 0.2985
Epoch 17/100, Train Loss: 0.2732, Val Loss: 0.2766
Epoch 18/100, Train Loss: 0.2486, Val Loss: 0.2533
Epoch 19/100, Train Loss: 0.2286, Val Loss: 0.2313
Epoch 20/100, Train Loss: 0.2057, Val Lo

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/100, Train Loss: 0.4028, Val Loss: 0.3431
Epoch 2/100, Train Loss: 0.3877, Val Loss: 0.3347
Epoch 3/100, Train Loss: 0.3766, Val Loss: 0.3263
Epoch 4/100, Train Loss: 0.3642, Val Loss: 0.3181
Epoch 5/100, Train Loss: 0.3594, Val Loss: 0.3097
Epoch 6/100, Train Loss: 0.3495, Val Loss: 0.3013
Epoch 7/100, Train Loss: 0.3414, Val Loss: 0.2929
Epoch 8/100, Train Loss: 0.3318, Val Loss: 0.2842
Epoch 9/100, Train Loss: 0.3258, Val Loss: 0.2754
Epoch 10/100, Train Loss: 0.3127, Val Loss: 0.2663
Epoch 11/100, Train Loss: 0.3048, Val Loss: 0.2567
Epoch 12/100, Train Loss: 0.2970, Val Loss: 0.2470
Epoch 13/100, Train Loss: 0.2884, Val Loss: 0.2367
Epoch 14/100, Train Loss: 0.2770, Val Loss: 0.2257
Epoch 15/100, Train Loss: 0.2559, Val Loss: 0.2142
Epoch 16/100, Train Loss: 0.2438, Val Loss: 0.2028
Epoch 17/100, Train Loss: 0.2307, Val Loss: 0.1922
Epoch 18/100, Train Loss: 0.2137, Val Loss: 0.1824
Epoch 19/100, Train Loss: 0.2019, Val Loss: 0.1725
Epoch 20/100, Train Loss: 0.1877, Val Lo

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

# Prepare Submission File

In [8]:
sample_submission = pd.read_csv('./sample_submission.csv')

for 품목명, predictions in 품목별_predictions.items():
    sample_submission[품목명] = predictions

# 결과 저장
sample_submission.to_csv('./baseline_submission6.csv', index=False)
